In [49]:
import streamlit as st
from openpyxl import load_workbook
import pandas as pd
import plotly.graph_objs as go

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/wind_speed_laurel_nebraska.csv')

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['Time'],
        y=df['10 Min Sampled Avg'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=df['Time'],
        y=df['10 Min Sampled Avg']+df['10 Min Std Dev'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=df['Time'],
        y=df['10 Min Sampled Avg']-df['10 Min Std Dev'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Wind speed (m/s)',
    title='Continuous, variable value error bars',
    hovermode="x"
)
fig.show()


In [50]:
import streamlit as st
from openpyxl import load_workbook
import pandas as pd
import plotly.graph_objs as go


data_file = "./DB/All-at-once_DB.xlsx"
df_SR_original = pd.read_excel(data_file, header = 0)
df_SR_original.head()

,Fissile,Case,Code,Library,Flux-to-dose conversion factor,Screen,Thickness (cm),Particle,Distance (m),Dose (Gy),1s uncertainty
0,U,C1 [U(4.95)O2F2 (H/235U = 410)],MCNP 6.1,ENDF/B-VII.1,ANSI/HPS N13.3,Concrete,20,N,1,2.457272,0.0225
1,U,C1 [U(4.95)O2F2 (H/235U = 410)],MCNP 6.1,ENDF/B-VII.1,ANSI/HPS N13.3,Concrete,20,N,2,0.772401,0.0084
2,U,C1 [U(4.95)O2F2 (H/235U = 410)],MCNP 6.1,ENDF/B-VII.1,ANSI/HPS N13.3,Concrete,20,N,5,0.143430,0.0097
3,U,C1 [U(4.95)O2F2 (H/235U = 410)],MCNP 6.1,ENDF/B-VII.1,ANSI/HPS N13.3,Concrete,20,N,10,0.035122,0.0093
4,U,C1 [U(4.95)O2F2 (H/235U = 410)],MCNP 6.1,ENDF/B-VII.1,ANSI/HPS N13.3,Concrete,20,N,20,0.008329,0.0071


In [51]:
to_filter_columns = df_SR_original.columns
to_filter_columns

Index(['Fissile', 'Case', 'Code', 'Library', 'Flux-to-dose conversion factor',
       'Screen', 'Thickness (cm)', 'Particle', 'Distance (m)', 'Dose (Gy)',
       '1s uncertainty'],
      dtype='object')

In [52]:
for column in to_filter_columns:
    print(column)

Fissile
Case
Code
Library
Flux-to-dose conversion factor
Screen
Thickness (cm)
Particle
Distance (m)
Dose (Gy)
1s uncertainty


In [53]:
df_SR_original['Screen'].unique()

array(['Concrete', 'Water', 'Lead', 'Steel', 'None'], dtype=object)

In [54]:
resultat_filtre = df_SR_original[df_SR_original['Screen'].str.contains('Concrete')]
resultat_filtre.shape

(312, 11)

In [55]:
print(df_SR_original[col]

SyntaxError: incomplete input (4248372471.py, line 1)

In [ ]:
def dataframe_multiselect_filters(df: pd.DataFrame) -> (pd.DataFrame, dict):
    # """
    # Adds a UI on top of a dataframe to let viewers filter columns

    # Args:
    #     df (pd.DataFrame): Original dataframe

    # Returns:
    #     pd.DataFrame: Filtered dataframe
    # """
    # # modify = st.checkbox("Add filters")

    # # if not modify:
    # #     return df

    # df = df.copy()

    # # Try to convert datetimes into a standard format (datetime, no timezone)
    # for col in df.columns:
    #     if is_object_dtype(df[col]):
    #         try:
    #             df[col] = pd.to_datetime(df[col])
    #         except Exception:
    #             pass

    #     if is_datetime64_any_dtype(df[col]):
    #         df[col] = df[col].dt.tz_localize(None)
    
    
    filters = {}  # Variable pour stocker les filtres sélectionnés

    modification_container = st.container()

    with modification_container:
        to_filter_column = st.selectbox("Filter dataframe on", df.columns, index=0)
        to_filter_columns = [to_filter_column] if to_filter_column else list(df.columns)
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            left.write("↳")
            # Treat columns with < 10 unique values as categorical
            if is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    df[column].unique(),
                    default=list(df[column].unique()),
                    #max_selections = 1
                )
                filters[column] = user_cat_input  # Stocke la valeur du filtre pour cette colonne
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    _min,
                    _max,
                    (_min, _max),
                    step=step,
                )
                filters[column] = user_num_input  # Stocke la valeur du filtre pour cette colonne
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Values for {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    filters[column] = user_date_input  # Stocke la valeur du filtre pour cette colonne
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    filters[column] = user_text_input  # Stocke la valeur du filtre pour cette colonne
                    df = df[df[column].str.contains(user_text_input)]

    return df, filters  # Retourne à la fois le DataFrame filtré et le dictionnaire de filtres

In [ ]:
filtered_SR_df, SR_filters = dataframe_multiselect_filters(df_SR_original)
# Affichage des filtres sélectionnés
st.write("Filtres sélectionnés :", SR_filters)

# Affichage du DataFrame filtré
st.write("DataFrame filtré :", filtered_SR_df) # = #st.dataframe(filtered_SR_df)


2023-11-20 16:07:12.641 
  command:

    streamlit run C:\Users\herth-joh01\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]


NameError: name 'is_categorical_dtype' is not defined

In [ ]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Convertir la colonne Thickness en type catégorie
#data['Thickness (cm)'] = data['Thickness (cm)'].astype('category')

# Fonction pour mettre à jour le graphique
def plot_filters(particle_type, screen_type, code_type, case_type):
    filtered_data = data[(data['Particle'] == particle_type) & 
                         (data['Screen'] == screen_type) &
                         (data['Code'] == code_type) &
                         (data['Case'] == case_type)]
    
    fig = px.scatter(filtered_data, x="Distance (m)", y="Dose (Gy)",
                     error_y="Absolute Uncertainty", error_y_minus="Absolute Uncertainty",
                     log_x=True, log_y=True,
                     title=f"Dose en fonction de la Distance (échelles logarithmiques)",
                     labels={"Distance (m)": "Distance (m) [Log]", "Dose (Gy)": "Dose (Gy) [Log]"},
                     color="Thickness (cm)", 
                     hover_data=['Particle', 'Screen', 'Code', 'Case'])

    fig.update_layout(legend_title_text='Épaisseur (cm)')
    fig.show()

# Création des widgets de sélection
particle_selector = widgets.Dropdown(
    options=list(data['Particle'].unique()),
    value=list(data['Particle'].unique())[0],
    description='Particule:',
)

screen_selector = widgets.Dropdown(
    options=list(data['Screen'].unique()),
    value=list(data['Screen'].unique())[0],
    description='Écran:',
)

code_selector = widgets.Dropdown(
    options=list(data['Code'].unique()),
    value=list(data['Code'].unique())[0],
    description='Code:',
)

case_selector = widgets.Dropdown(
    options=list(data['Case'].unique()),
    value=list(data['Case'].unique())[0],
    description='Cas:',
)

# Afficher les widgets et le graphique
widgets.interactive(plot_filters, 
                    particle_type=particle_selector, 
                    screen_type=screen_selector, 
                    code_type=code_selector,
                    case_type=case_selector)


In [15]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Convertir la colonne Thickness en type catégorie
data['Thickness (cm)'] = data['Thickness (cm)'].astype('category')

# Créer un graphique Plotly de base sans widgets ni filtrage
fig = px.scatter(data, x="Distance (m)", y="Dose (Gy)",
                 error_y="Absolute Uncertainty", error_y_minus="Absolute Uncertainty",
                 log_x=True, log_y=True,
                 title="Dose en fonction de la Distance (échelles logarithmiques)",
                 labels={"Distance (m)": "Distance (m) [Log]", "Dose (Gy)": "Dose (Gy) [Log]"},
                 hover_data=['Particle', 'Screen', 'Code', 'Case'],
                 color="Thickness (cm)")

fig.update_xaxes(type="log", showgrid=True, title="Distance (m)")
fig.update_xaxes(minor = dict(ticks="inside", ticklen=6, tickcolor='white', griddash='dot', showgrid=True))
fig.update_yaxes(type='log', showgrid=True, tickformat='.2e',  title="Dose (Gy)")
fig.update_yaxes(minor = dict(ticks="inside", ticklen=6, tickcolor='white', griddash='dot', showgrid=True))

fig.update_layout(legend_title_text='Épaisseur (cm)')
fig.show()

In [31]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Fonction pour mettre à jour le graphique
def plot_filters(particle_types, screen_types, code_types, case_types):
    filtered_data = data[data['Particle'].isin(particle_types) & 
                         data['Screen'].isin(screen_types) &
                         data['Code'].isin(code_types) &
                         data['Case'].isin(case_types)]
    
    fig = px.scatter(filtered_data, x="Distance (m)", y="Dose (Gy)",
                     error_y="Absolute Uncertainty", error_y_minus="Absolute Uncertainty",
                     log_x=True, log_y=True,
                     title="Dose en fonction de la Distance (échelles logarithmiques)",
                     labels={"Distance (m)": "Distance (m) [Log]", "Dose (Gy)": "Dose (Gy) [Log]"},
                     color="Thickness (cm)", 
                     hover_data=['Particle', 'Screen', 'Code', 'Case'])

    fig.update_layout(legend_title_text='Épaisseur (cm)')
    fig.show()

# Création des widgets de sélection multiple
particle_selector = widgets.SelectMultiple(
    options=list(data['Particle'].unique()),
    value=[data['Particle'].unique()[0]],
    description='Particule:',
)

screen_selector = widgets.SelectMultiple(
    options=list(data['Screen'].unique()),
    value=[data['Screen'].unique()[0]],
    description='Écran:',
)

code_selector = widgets.SelectMultiple(
    options=list(data['Code'].unique()),
    value=[data['Code'].unique()[0]],
    description='Code:',
)

case_selector = widgets.SelectMultiple(
    options=list(data['Case'].unique()),
    value=[data['Case'].unique()[0]],
    description='Cas:',
)

# Afficher les widgets et le graphique
interactive_plot = widgets.interactive(plot_filters, 
                    particle_types=particle_selector, 
                    screen_types=screen_selector, 
                    code_types=code_selector,
                    case_types=case_selector)

display(interactive_plot)


interactive(children=(SelectMultiple(description='Particule:', index=(0,), options=('N', 'P'), value=('N',)), …

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Fonction pour mettre à jour le graphique
def plot_filters(particle_types, screen_types, code_types, case_types):
    filtered_data = data[data['Particle'].isin(particle_types) & 
                         data['Screen'].isin(screen_types) &
                         data['Code'].isin(code_types) &
                         data['Case'].isin(case_types)]
    
    # Créer un graphique de type scatter
    fig = px.scatter(filtered_data, x="Distance (m)", y="Dose (Gy)",
                     error_y="Absolute Uncertainty", error_y_minus="Absolute Uncertainty",
                     log_x=True, log_y=True,
                     title="Dose en fonction de la Distance (échelles logarithmiques)",
                     labels={"Distance (m)": "Distance (m) [Log]", "Dose (Gy)": "Dose (Gy) [Log]"},
                     color="Thickness (cm)", 
                     hover_data=['Particle', 'Screen', 'Code', 'Case'])

    # Ajouter des lignes pointillées pour chaque groupe de couleur
    for name, group in filtered_data.groupby("Thickness (cm)"):
        fig.add_trace(go.Scatter(x=group["Distance (m)"], y=group["Dose (Gy)"], 
                                 mode='lines', name=name, line=dict(dash='dash')))

    fig.update_layout(legend_title_text='Épaisseur (cm)')
    fig.show()

# Création des widgets de sélection multiple
particle_selector = widgets.SelectMultiple(
    options=list(data['Particle'].unique()),
    value=[data['Particle'].unique()[0]],
    description='Particule:',
)

screen_selector = widgets.SelectMultiple(
    options=list(data['Screen'].unique()),
    value=[data['Screen'].unique()[0]],
    description='Écran:',
)

code_selector = widgets.SelectMultiple(
    options=list(data['Code'].unique()),
    value=[data['Code'].unique()[0]],
    description='Code:',
)

case_selector = widgets.SelectMultiple(
    options=list(data['Case'].unique()),
    value=[data['Case'].unique()[0]],
    description='Cas:',
)

# Afficher les widgets et le graphique
interactive_plot = widgets.interactive(plot_filters, 
                    particle_types=particle_selector, 
                    screen_types=screen_selector, 
                    code_types=code_selector,
                    case_types=case_selector)

display(interactive_plot)


In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Ajouter une colonne pour la combinaison de filtres
data['Filter Combo'] = data.apply(lambda row: f"{row['Particle']}_{row['Screen']}_{row['Code']}_{row['Case']}_{row['Thickness (cm)']}", axis=1)

# Fonction pour mettre à jour le graphique
def plot_filters(particle_types, screen_types, code_types, case_types, thickness_types):
    filtered_data = data[data['Particle'].isin(particle_types) & 
                         data['Screen'].isin(screen_types) &
                         data['Code'].isin(code_types) &
                         data['Case'].isin(case_types) &
                         data['Thickness (cm)'].isin(thickness_types)]
    
    fig = px.scatter(filtered_data, x="Distance (m)", y="Dose (Gy)",
                     error_y="Absolute Uncertainty", error_y_minus="Absolute Uncertainty",
                     log_x=True, log_y=True,
                     title="Dose en fonction de la Distance (échelles logarithmiques)",
                     labels={"Distance (m)": "Distance (m) [Log]", "Dose (Gy)": "Dose (Gy) [Log]"},
                     color="Filter Combo", 
                     hover_data=['Particle', 'Screen', 'Code', 'Case', 'Thickness (cm)'])

    fig.update_layout(legend_title_text='Combinaison des Filtres')
    fig.show()

# Création des widgets de sélection multiple
particle_selector = widgets.SelectMultiple(
    options=list(data['Particle'].unique()),
    value=[data['Particle'].unique()[0]],
    description='Particule:',
)

screen_selector = widgets.SelectMultiple(
    options=list(data['Screen'].unique()),
    value=[data['Screen'].unique()[0]],
    description='Écran:',
)

code_selector = widgets.SelectMultiple(
    options=list(data['Code'].unique()),
    value=[data['Code'].unique()[0]],
    description='Code:',
)

case_selector = widgets.SelectMultiple(
    options=list(data['Case'].unique()),
    value=[data['Case'].unique()[0]],
    description='Cas:',
)

thickness_selector = widgets.SelectMultiple(
    options=list(data['Thickness (cm)'].unique()),
    value=[data['Thickness (cm)'].unique()[0]],
    description='Épaisseur:',
)

# Afficher les widgets et le graphique
interactive_plot = widgets.interactive(plot_filters, 
                    particle_types=particle_selector, 
                    screen_types=screen_selector, 
                    code_types=code_selector,
                    case_types=case_selector,
                    thickness_types=thickness_selector)

display(interactive_plot)


interactive(children=(SelectMultiple(description='Particule:', index=(0,), options=('N', 'P'), value=('N',)), …

In [4]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Charger vos données
data = pd.read_excel("./DB/All-at-once_DB.xlsx")

# Calcul des incertitudes absolues
data["Absolute Uncertainty"] = (data["1s uncertainty"] / 100) * data["Dose (Gy)"]

# Ajouter une colonne pour la combinaison de filtres
data['Filter Combo'] = data.apply(lambda row: f"{row['Particle']}_{row['Screen']}_{row['Code']}_{row['Case']}_{row['Thickness (cm)']}", axis=1)

# Fonction pour mettre à jour le graphique
def plot_filters(particle_types, screen_types, code_types, case_types, thickness_types):
    filtered_data = data[data['Particle'].isin(particle_types) & 
                         data['Screen'].isin(screen_types) &
                         data['Code'].isin(code_types) &
                         data['Case'].isin(case_types) &
                         data['Thickness (cm)'].isin(thickness_types)]

    # Créer une figure avec Plotly Graph Objects
    fig = go.Figure()

    # Ajouter des traces pour chaque combinaison de filtres
    for combo in filtered_data['Filter Combo'].unique():
        df_subset = filtered_data[filtered_data['Filter Combo'] == combo]
        fig.add_trace(go.Scatter(x=df_subset["Distance (m)"], y=df_subset["Dose (Gy)"],
                                 mode='lines+markers', name=combo,
                                 line=dict(dash='dash'),
                                 error_y=dict(type='data', array=df_subset["Absolute Uncertainty"],
                                              visible=True)))

    fig.update_layout(
        title="Dose en fonction de la Distance (échelles logarithmiques)",
        xaxis_title="Distance (m) [Log]",
        yaxis_title="Dose (Gy) [Log]",
        legend_title="Combinaison des Filtres",
        xaxis={'type': 'log'},
        yaxis={'type': 'log'}
    )
    fig.show()

# Création des widgets de sélection multiple
particle_selector = widgets.SelectMultiple(
    options=list(data['Particle'].unique()),
    value=[data['Particle'].unique()[0]],
    description='Particule:',
)

screen_selector = widgets.SelectMultiple(
    options=list(data['Screen'].unique()),
    value=[data['Screen'].unique()[0]],
    description='Écran:',
)

code_selector = widgets.SelectMultiple(
    options=list(data['Code'].unique()),
    value=[data['Code'].unique()[0]],
    description='Code:',
)

case_selector = widgets.SelectMultiple(
    options=list(data['Case'].unique()),
    value=[data['Case'].unique()[0]],
    description='Cas:',
)

thickness_selector = widgets.SelectMultiple(
    options=list(data['Thickness (cm)'].unique()),
    value=[data['Thickness (cm)'].unique()[0]],
    description='Épaisseur:',
)

# Afficher les widgets et le graphique
interactive_plot = widgets.interactive(plot_filters, 
                    particle_types=particle_selector, 
                    screen_types=screen_selector, 
                    code_types=code_selector,
                    case_types=case_selector,
                    thickness_types=thickness_selector)

display(interactive_plot)


interactive(children=(SelectMultiple(description='Particule:', index=(0,), options=('N', 'P'), value=('N',)), …